In [1]:
import os
os.getcwd()

'/Users/gracewhyn/Documents/KULIAH/PROJECTS/penyisihan-datavidia-10'

In [2]:
# Load and inspect the provided preprocessed ISPU dataset
import pandas as pd

df = pd.read_csv("ispu_preprocessed.csv")

# Basic inspection outputs
print("Shape:", df.shape)
print("\nColumns:")
print(df.columns.tolist())

print("\nInfo:")
print(df.info())

print("\nMissing values per column:")
print(df.isna().sum().sort_values(ascending=False).head(20))

print("\nSample rows:")
print(df.head())


Shape: (15356, 12)

Columns:
['periode_data', 'tanggal', 'stasiun', 'pm_sepuluh', 'pm_duakomalima', 'sulfur_dioksida', 'karbon_monoksida', 'ozon', 'nitrogen_dioksida', 'max', 'parameter_pencemar_kritis', 'kategori']

Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15356 entries, 0 to 15355
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   periode_data               15356 non-null  int64  
 1   tanggal                    15356 non-null  object 
 2   stasiun                    15356 non-null  object 
 3   pm_sepuluh                 14648 non-null  float64
 4   pm_duakomalima             6579 non-null   float64
 5   sulfur_dioksida            15058 non-null  float64
 6   karbon_monoksida           15151 non-null  float64
 7   ozon                       15014 non-null  float64
 8   nitrogen_dioksida          15047 non-null  float64
 9   max                        15356 non-null  floa

In [11]:
# Deeper missing value summary and basic stats for pollutant columns
pollutant_cols = [
    "pm10",
    "pm25",
    "so2",
    "co",
    "o3",
    "no2"
]

print("\nMissing values (pollutants):")
print(df[pollutant_cols].isna().sum())

print("\nBasic statistics (pollutants):")
print(df[pollutant_cols].describe())

# Check unique categories and stations
print("\nUnique kategori:", df['kategori'].unique())
print("Unique stasiun:", df['stasiun'].unique())



Missing values (pollutants):
pm10    0
pm25    0
so2     0
co      0
o3      0
no2     0
dtype: int64

Basic statistics (pollutants):
               pm10          pm25           so2            co            o3  \
count  15356.000000  15356.000000  15356.000000  15356.000000  15356.000000   
mean      53.976556     73.522662     27.695428     20.263089     53.407137   
std       18.902109     21.447810     15.642490     12.256185     40.360073   
min        2.000000     10.000000      0.000000      0.000000      2.000000   
25%       42.000000     60.000000     15.000000     11.000000     25.000000   
50%       55.000000     67.000000     26.000000     18.000000     42.000000   
75%       65.000000     89.000000     38.000000     26.000000     71.000000   
max      187.000000    287.000000    112.000000    134.000000    314.000000   

                no2  
count  15356.000000  
mean      17.117153  
std       11.655028  
min        0.000000  
25%        9.000000  
50%       15.000000  

In [10]:
df['tanggal'] = pd.to_datetime(df['tanggal'])
df = df.sort_values(['stasiun','tanggal'])

for col in pollutant_cols:
    df[col] = df.groupby('stasiun')[col].transform(lambda x: x.ffill().bfill())

mask_pm25 = df['pm25'].isna()

df.loc[mask_pm25, 'pm25'] = (
    df.loc[mask_pm25, 'pm10'] * 0.6
)

for col in pollutant_cols:
    df[col] = df.groupby('stasiun')[col].transform(lambda x: x.fillna(x.median()))


In [7]:
df

,periode_data,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,parameter_pencemar_kritis,kategori
0,201001,2010-01-01,DKI1,60.0,NaN,4.0,73.0,27.0,14.0,73.0,CO,SEDANG
1,201001,2010-01-02,DKI1,32.0,NaN,2.0,16.0,33.0,9.0,33.0,O3,BAIK
2,201001,2010-01-03,DKI1,27.0,NaN,2.0,19.0,20.0,9.0,27.0,PM10,BAIK
3,201001,2010-01-04,DKI1,22.0,NaN,2.0,16.0,15.0,6.0,22.0,PM10,BAIK
4,201001,2010-01-05,DKI1,25.0,NaN,2.0,17.0,15.0,8.0,25.0,PM10,BAIK
...,...,...,...,...,...,...,...,...,...,...,...,...
15351,202508,2025-08-31,DKI1,42.0,70.0,29.0,12.0,15.0,24.0,70.0,PM25,SEDANG
15352,202508,2025-08-31,DKI2,NaN,72.0,45.0,16.0,21.0,16.0,72.0,PM25,SEDANG
15353,202508,2025-08-31,DKI3,28.0,60.0,53.0,8.0,19.0,39.0,60.0,PM25,SEDANG
15354,202508,2025-08-31,DKI4,47.0,59.0,27.0,10.0,18.0,17.0,59.0,PM25,SEDANG


In [12]:
df

,periode_data,tanggal,stasiun,pm10,pm25,so2,co,o3,no2,max,parameter_pencemar_kritis,kategori
0,201001,2010-01-01,DKI1,60.0,51.0,4.0,73.0,27.0,14.0,73.0,CO,SEDANG
1,201001,2010-01-02,DKI1,32.0,51.0,2.0,16.0,33.0,9.0,33.0,O3,BAIK
2,201001,2010-01-03,DKI1,27.0,51.0,2.0,19.0,20.0,9.0,27.0,PM10,BAIK
3,201001,2010-01-04,DKI1,22.0,51.0,2.0,16.0,15.0,6.0,22.0,PM10,BAIK
4,201001,2010-01-05,DKI1,25.0,51.0,2.0,17.0,15.0,8.0,25.0,PM10,BAIK
...,...,...,...,...,...,...,...,...,...,...,...,...
15335,202508,2025-08-27,DKI5,57.0,110.0,25.0,11.0,84.0,46.0,110.0,PM25,TIDAK SEHAT
15340,202508,2025-08-28,DKI5,32.0,68.0,25.0,7.0,31.0,28.0,68.0,PM25,SEDANG
15345,202508,2025-08-29,DKI5,27.0,60.0,24.0,6.0,37.0,27.0,60.0,PM25,SEDANG
15350,202508,2025-08-30,DKI5,38.0,72.0,26.0,9.0,30.0,40.0,72.0,PM25,SEDANG


In [13]:
print("Shape:", df.shape)
print("\nColumns:")
print(df.columns.tolist())

print("\nInfo:")
print(df.info())

print("\nMissing values per column:")
print(df.isna().sum().sort_values(ascending=False).head(20))

print("\nSample rows:")
print(df.head())

Shape: (15356, 12)

Columns:
['periode_data', 'tanggal', 'stasiun', 'pm10', 'pm25', 'so2', 'co', 'o3', 'no2', 'max', 'parameter_pencemar_kritis', 'kategori']

Info:
<class 'pandas.core.frame.DataFrame'>
Index: 15356 entries, 0 to 15355
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   periode_data               15356 non-null  int64         
 1   tanggal                    15356 non-null  datetime64[ns]
 2   stasiun                    15356 non-null  object        
 3   pm10                       15356 non-null  float64       
 4   pm25                       15356 non-null  float64       
 5   so2                        15356 non-null  float64       
 6   co                         15356 non-null  float64       
 7   o3                         15356 non-null  float64       
 8   no2                        15356 non-null  float64       
 9   max                        15356 

In [5]:
df = df.rename(columns={
    "pm_sepuluh": "pm10",
    "pm_duakomalima": "pm25",
    "sulfur_dioksida": "so2",
    "karbon_monoksida": "co",
    "ozon": "o3",
    "nitrogen_dioksida": "no2"
})


In [33]:
# encode stasiun 
df["stasiun"] = df["stasiun"].astype(str)


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15356 entries, 0 to 15355
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   periode_data               15356 non-null  int64         
 1   tanggal                    15356 non-null  datetime64[ns]
 2   stasiun                    15356 non-null  object        
 3   pm10                       15356 non-null  float64       
 4   pm25                       15356 non-null  float64       
 5   so2                        15356 non-null  float64       
 6   co                         15356 non-null  float64       
 7   o3                         15356 non-null  float64       
 8   no2                        15356 non-null  float64       
 9   max                        15356 non-null  float64       
 10  parameter_pencemar_kritis  15312 non-null  object        
 11  kategori                   15356 non-null  object        
dtypes: dateti

In [15]:
df.to_csv("ispu_clean.csv", index=False)

In [26]:
import pandas as pd

df = pd.read_csv("ispu_clean_fe.csv", parse_dates=["tanggal"])

df = df.sort_values("tanggal")

split_date = df["tanggal"].quantile(0.8)

train = df[df["tanggal"] <= split_date]
test  = df[df["tanggal"] > split_date]


In [27]:
drop_cols = [
    "tanggal",
    "kategori",
    "parameter_pencemar_kritis",
    "calc_max",
    "periode_data",
    "glitch_pm10",
    "glitch_pm25"
]

features = [c for c in df.columns if c not in drop_cols]

X_train = train[features]
y_train = train["kategori"]

X_test = test[features]
y_test = test["kategori"]


In [69]:
X_train

,stasiun,pm10_jump,pm25_jump,glitch_pm10,glitch_pm25,day,month,dayofweek,weekofyear,pm10_lag1,...,o3_roll3,o3_roll7,no2_roll3,no2_roll7,pm10_rollmax7,pm25_rollmax7,so2_rollmax7,co_rollmax7,o3_rollmax7,no2_rollmax7
0,DKI1,23.0,0.0,False,False,8,1,4,1,41.0,...,21.666667,21.142857,13.000000,10.142857,64.0,51.0,8.0,51.0,33.0,15.0
1,DKI1,9.0,0.0,False,False,9,1,5,1,64.0,...,23.000000,19.714286,14.666667,11.142857,64.0,51.0,8.0,51.0,27.0,16.0
2,DKI1,21.0,0.0,False,False,10,1,6,1,55.0,...,22.666667,20.571429,14.333333,11.571429,64.0,51.0,8.0,51.0,27.0,16.0
3,DKI1,21.0,0.0,False,False,11,1,0,2,34.0,...,24.000000,21.714286,14.666667,13.000000,64.0,51.0,10.0,51.0,27.0,16.0
4,DKI1,10.0,0.0,False,False,12,1,1,2,55.0,...,19.333333,20.857143,15.000000,14.285714,64.0,51.0,10.0,51.0,27.0,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8359,DKI3,19.0,1.0,False,False,30,11,3,48,56.0,...,21.000000,21.000000,11.333333,14.571429,66.0,101.0,57.0,13.0,27.0,21.0
2246,DKI1,20.0,15.0,False,False,30,11,3,48,58.0,...,31.333333,25.285714,31.333333,30.285714,70.0,101.0,45.0,15.0,34.0,38.0
5391,DKI2,24.0,19.0,False,False,30,11,3,48,62.0,...,29.000000,27.571429,27.000000,31.571429,72.0,99.0,36.0,15.0,34.0,47.0
14669,DKI5,26.0,31.0,False,False,30,11,3,48,56.0,...,55.000000,52.142857,18.666667,20.428571,71.0,112.0,36.0,23.0,61.0,26.0


In [36]:
from catboost import CatBoostClassifier

drop_cols = ["tanggal", "kategori", "parameter_pencemar_kritis", "periode_data"]

features = [c for c in df.columns if c not in drop_cols]

X_train = train[features]
y_train = train["kategori"]

X_test = test[features]
y_test = test["kategori"]

model = CatBoostClassifier(
    iterations=1000,
    depth=6,
    learning_rate=0.05,
    loss_function='MultiClass',
    verbose=200
)

model.fit(X_train, y_train, cat_features=["stasiun"])

from sklearn.metrics import classification_report
print(classification_report(y_test, model.predict(X_test)))


0:	learn: 1.3095424	total: 11.6ms	remaining: 11.6s
200:	learn: 0.3936520	total: 1.42s	remaining: 5.65s
400:	learn: 0.3347517	total: 2.73s	remaining: 4.07s
600:	learn: 0.3014145	total: 3.99s	remaining: 2.65s
800:	learn: 0.2755289	total: 5.31s	remaining: 1.32s
999:	learn: 0.2540854	total: 6.65s	remaining: 0us
              precision    recall  f1-score   support

        BAIK       0.65      0.63      0.64       361
      SEDANG       0.89      0.94      0.91      2303
 TIDAK SEHAT       0.84      0.52      0.65       319

    accuracy                           0.86      2983
   macro avg       0.79      0.70      0.73      2983
weighted avg       0.85      0.86      0.85      2983



In [40]:
sample = pd.read_csv("sample_submission.csv")
sample.head()


,id,category
0,2025-09-01_DKI1,NaN
1,2025-09-01_DKI2,NaN
2,2025-09-01_DKI3,NaN
3,2025-09-01_DKI4,NaN
4,2025-09-01_DKI5,NaN
